### Penyesuaian Data Gambar

In [1]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import os
from PIL import Image
import shutil


import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50, ResNet101, InceptionV3

pd.set_option('display.max_column', None)

In [2]:
# Read data list_attribute
df = pd.read_csv('list_attribute.csv')
df.shape

(202599, 41)

In [3]:
df.head()

,Nama_File,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,-1,1,-1,-1,1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,1.0
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1.0
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1.0
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,1,-1,1,-1,1,1,-1,1.0
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1.0


In [4]:
# Read data class_identity
idn = pd.read_csv('class_identity.txt', sep=' ', header=None)
idn.shape

(202599, 2)

In [5]:
idn.head()

,0,1
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692
3,000004.jpg,5805
4,000005.jpg,9295


In [6]:
# Cek jumlah cewe dan cowo
df['Male'].value_counts()

Male
-1    118165
 1     84434
Name: count, dtype: int64

In [7]:
# Cek jumlah data di dataset csv
file_list = df['Nama_File'].to_list()
len(file_list)

202599

In [8]:
# Cek jumlah data di folder images
folder_list = os.listdir('Images')
len(folder_list)

5017

In [9]:
# Cek kesesuaian jumlah data di folder images dan di dataset
set_file = set(file_list)
set_folder = set(folder_list)
ada = []
for i in set_file:
  if i in set_folder:
    ada.append(i)
print(len(ada))

5000


In [10]:
# Cek 17 data yang tidak ada pada dataset
tmp = os.listdir('Images')
tidak_ada = []
for i in tmp:
    if i not in ada:
      tidak_ada.append(i)
print(tidak_ada)

['182793(1).jpg', '182809(1).jpg', '182912(1).jpg', '182943(1).jpg', '183005(1).jpg', '183018(1).jpg', '183050(1).jpg', '183111(1).jpg', '183121(1).jpg', '183145(1).jpg', '189132(1).jpg', '189297(1).jpg', '189324(1).jpg', '189512(1).jpg', '189513(1).jpg', '189581(1).jpg', '189651(1).jpg']


In [11]:
# Membuat dataframe baru untuk 5000 data (semua fitur)
df_final = df[df['Nama_File'].isin(ada)].reset_index(drop=True)
df_final.shape

## Kalau mau disave
# df_final.to_csv('df_final.csv')

(5000, 41)

In [12]:
df_final.head()

,Nama_File,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000051.jpg,1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0
1,000052.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1.0
2,000065.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,1.0
3,000166.jpg,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1.0
4,000198.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1.0


In [13]:
# Membuat dataframe baru untuk 5000 data (hanya jenis kelamin)
tmp = df_final.copy()
tmp['Gender'] = np.where(tmp['Male']==-1, 'Female', 'Male')
df_final_gender = tmp[['Nama_File', 'Male', 'Gender']]
df_final_gender['Male'] = df_final_gender['Male'].replace({-1: 0, 1: 1})
df_final_gender.shape

## Kalau mau disave
# df_final_gender.to_csv('df_final_gender.csv')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6504\705121905.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_gender['Male'] = df_final_gender['Male'].replace({-1: 0, 1: 1})


(5000, 3)

In [14]:
df_final_gender.head()

,Nama_File,Male,Gender
0,000051.jpg,1,Male
1,000052.jpg,1,Male
2,000065.jpg,1,Male
3,000166.jpg,1,Male
4,000198.jpg,0,Female


In [15]:
df_final_gender['Male'].value_counts()

Male
0    2953
1    2047
Name: count, dtype: int64

### Memisahkan Kelas pada Data Gambar

In [16]:
data_images = 'Images'

In [17]:
# Mengambil data laki-laki dan perempuan
laki = df_final_gender[df_final_gender['Male'] == 1].sample(n=2047, random_state=42) # 1637 untuk train dan 410 untuk validation (80:20)
perempuan = df_final_gender[df_final_gender['Male'] == 0].sample(n=2047, random_state=42) # 1637 untuk train dan 410 untuk validation (80:20)

In [18]:
# Split menjadi data train dan validation
laki_train = laki.iloc[:1637]
laki_val = laki.iloc[1637:]
perempuan_train = perempuan.iloc[:1637]
perempuan_val = perempuan.iloc[1637:]

In [19]:
# Direktori untuk data train dan validation
laki_train_dir = os.path.join(data_images, 'train/male')
perempuan_train_dir = os.path.join(data_images, 'train/female')
laki_val_dir = os.path.join(data_images, 'validation/male')
perempuan_val_dir = os.path.join(data_images, 'validation/female')

# Membuat direktori
os.makedirs(laki_train_dir, exist_ok=True)
os.makedirs(perempuan_train_dir, exist_ok=True)
os.makedirs(laki_val_dir, exist_ok=True)
os.makedirs(perempuan_val_dir, exist_ok=True)

In [20]:
# Memasukkan gambar ke direktori yang telah dibuat
def data_baru(list_gambar, folder_gambar, folder_tujuan):
    for gambar in list_gambar:
        src_file = os.path.join(folder_gambar, gambar)
        dest_file = os.path.join(folder_tujuan, gambar)
        shutil.copy(src_file, dest_file)

data_baru(laki_train['Nama_File'], data_images, laki_train_dir)
data_baru(perempuan_train['Nama_File'], data_images, perempuan_train_dir)
data_baru(laki_val['Nama_File'], data_images, laki_val_dir)
data_baru(perempuan_val['Nama_File'], data_images, perempuan_val_dir)

In [21]:
print(f"Jumlah data train laki adalah {len(os.listdir(laki_train_dir))}")
print(f"Jumlah data validasi laki adalah {len(os.listdir(laki_val_dir))}")
print(f"Jumlah data train perempuan adalah {len(os.listdir(perempuan_train_dir))}")
print(f"Jumlah data validasi perempuan adalah {len(os.listdir(perempuan_val_dir))}")

Jumlah data train laki adalah 1637
Jumlah data validasi laki adalah 410
Jumlah data train perempuan adalah 1637
Jumlah data validasi perempuan adalah 410


### Pemrosesan Data Gambar

In [22]:
# Direktori
base_dir = 'Images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [23]:
# Fungsi untuk augmentasi
def train_val_generators(TRAINING_DIR, VALIDATION_DIR, size):
   train_datagen = ImageDataGenerator(
      rescale=1./255.,
      rotation_range=20,
      zoom_range=0.2,
      shear_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
   )
   train_generator = train_datagen.flow_from_directory(
      directory=TRAINING_DIR,
      batch_size=32,
      target_size=size,
      class_mode='binary'
   )

   validation_datagen = ImageDataGenerator(
      rescale=1./255.
   )
   validation_generator = validation_datagen.flow_from_directory(
      directory=VALIDATION_DIR,
      batch_size=32,
      target_size=size,
      class_mode='binary'
   )
   return train_generator, validation_generator


In [24]:
# Augmentasi untuk VGG
train_generator_vgg, validation_generator_vgg = train_val_generators(train_dir, validation_dir, (224, 224))

# Augmentasi untuk ResNet
train_generator_res, validation_generator_res = train_val_generators(train_dir, validation_dir, (224, 224))

# Augmentasi untuk Inception
train_generator_inc, validation_generator_inc = train_val_generators(train_dir, validation_dir, (299, 299))

Found 3274 images belonging to 2 classes.
Found 820 images belonging to 2 classes.
Found 3274 images belonging to 2 classes.
Found 820 images belonging to 2 classes.
Found 3274 images belonging to 2 classes.
Found 820 images belonging to 2 classes.


### Modelling

###### VGG-16

In [25]:
# Modelling menggunakan VGG-16
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg16.layers:
  layer.trainable = False

x = vgg16.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

model_vgg16 = Model(inputs=vgg16.input, outputs=prediction)
model_vgg16.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [26]:
import time

# Compile dan fit model VGG16
model_vgg16.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

# Memulai waktu training
start_time = time.time()

# Training
history_vgg16 = model_vgg16.fit(
    train_generator_vgg,
    epochs=10,
    validation_data=validation_generator_vgg)

# Menghitung waktu training
end_time = time.time()
total_training_time = end_time - start_time
print(f"Total waktu training: {total_training_time:.2f} detik")

Epoch 1/10


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


103/103 ━━━━━━━━━━━━━━━━━━━━ 2514s 24s/step - accuracy: 0.7566 - loss: 0.8008 - val_accuracy: 0.9232 - val_loss: 0.2014
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2564s 25s/step - accuracy: 0.9056 - loss: 0.2380 - val_accuracy: 0.9134 - val_loss: 0.2205
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2494s 24s/step - accuracy: 0.9193 - loss: 0.2035 - val_accuracy: 0.9195 - val_loss: 0.2391
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2257s 22s/step - accuracy: 0.9288 - loss: 0.1963 - val_accuracy: 0.8610 - val_loss: 0.3884
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2284s 22s/step - accuracy: 0.9369 - loss: 0.1542 - val_accuracy: 0.9354 - val_loss: 0.1851
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2261s 22s/step - accuracy: 0.9534 - loss: 0.1319 - val_accuracy: 0.9366 - val_loss: 0.1840
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2249s 22s/step - accuracy: 0.9530 - loss: 0.1236 - val_accuracy: 0.9293 - val_loss: 0.1952
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 2099s 20s/step - accuracy: 0.9581 - loss: 0.1055 - val

In [ ]:
# import time

# # Compile dan fit model VGG16 (menggunakan optimizer Rmsprop)
# model_vgg16.compile(
#     loss='binary_crossentropy',
#     optimizer='rmsprop',
#     metrics=['accuracy']
#     )

# # Memulai waktu training
# start_time = time.time()

# # Training
# history_vgg16 = model_vgg16.fit(
#     train_generator_vgg,
#     epochs=10,
#     validation_data=validation_generator_vgg)

# # Menghitung waktu training
# end_time = time.time()
# total_training_time = end_time - start_time
# print(f"Total waktu training: {total_training_time:.2f} detik")

###### VGG-19

In [27]:
# Modelling menggunakan VGG-16
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg19.layers:
  layer.trainable = False

x = vgg19.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

model_vgg19 = Model(inputs=vgg19.input, outputs=prediction)
model_vgg19.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           25

 Total params: 26,447,425 (100.89 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [29]:
# Compile dan fit model VGG16
model_vgg19.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

# Memulai waktu training
start_time = time.time()

# Training
history_vgg19 = model_vgg19.fit(
    train_generator_vgg,
    epochs=10,
    validation_data=validation_generator_vgg)

# Menghitung waktu training
end_time = time.time()
total_training_time = end_time - start_time
print(f"Total waktu training: {total_training_time:.2f} detik")

In [ ]:
# # Compile dan fit model VGG16 (rmsprop)
# model_vgg19.compile(
#     loss='binary_crossentropy',
#     optimizer='rmsprop',
#     metrics=['accuracy']
#     )

# # Memulai waktu training
# start_time = time.time()

# # Training
# history_vgg19 = model_vgg19.fit(
#     train_generator_vgg,
#     epochs=10,
#     validation_data=validation_generator_vgg)

# # Menghitung waktu training
# end_time = time.time()
# total_training_time = end_time - start_time
# print(f"Total waktu training: {total_training_time:.2f} detik")

###### ResNet50

In [30]:
# Modelling menggunakan ResNet50
res50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in res50.layers:
  layer.trainable = False

x = res50.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

model_res50 = Model(inputs=res50.input, outputs=prediction)
model_res50.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 49,278,337 (187.98 MB)

 Trainable params: 25,690,625 (98.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [31]:
# Compile dan fit model ResNet50
model_res50.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

# Memulai waktu training
start_time = time.time()

# Training
history_res50 = model_res50.fit(
    train_generator_res,
    epochs=10,
    validation_data=validation_generator_res)

# Menghitung waktu training
end_time = time.time()
total_training_time = end_time - start_time
print(f"Total waktu training: {total_training_time:.2f} detik")

Epoch 1/10


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


103/103 ━━━━━━━━━━━━━━━━━━━━ 418s 4s/step - accuracy: 0.5337 - loss: 3.3913 - val_accuracy: 0.7732 - val_loss: 0.5098
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 497s 5s/step - accuracy: 0.5648 - loss: 1.7928 - val_accuracy: 0.7512 - val_loss: 0.4944
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 644s 6s/step - accuracy: 0.6666 - loss: 0.6641 - val_accuracy: 0.5878 - val_loss: 0.8157
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 719s 7s/step - accuracy: 0.7181 - loss: 0.5908 - val_accuracy: 0.7512 - val_loss: 0.4756
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 719s 7s/step - accuracy: 0.7382 - loss: 0.5400 - val_accuracy: 0.7695 - val_loss: 0.4620
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 716s 7s/step - accuracy: 0.7479 - loss: 0.5477 - val_accuracy: 0.8317 - val_loss: 0.4178
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 728s 7s/step - accuracy: 0.6341 - loss: 0.8178 - val_accuracy: 0.6232 - val_loss: 0.7653
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 710s 7s/step - accuracy: 0.7325 - loss: 0.5355 - val_accuracy: 0.763

In [ ]:
# # Compile dan fit model ResNet50 (rmsprop)
# model_res50.compile(
#     loss='binary_crossentropy',
#     optimizer='rmsprop',
#     metrics=['accuracy']
#     )

# # Memulai waktu training
# start_time = time.time()

# # Training
# history_res50 = model_res50.fit(
#     train_generator_res,
#     epochs=10,
#     validation_data=validation_generator_res)

# # Menghitung waktu training
# end_time = time.time()
# total_training_time = end_time - start_time
# print(f"Total waktu training: {total_training_time:.2f} detik")

###### ResNet101

In [32]:
# Modelling menggunakan ResNet101
res101 = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in res101.layers:
  layer.trainable = False

x = res101.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

model_res101 = Model(inputs=res101.input, outputs=prediction)
model_res101.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 68,348,801 (260.73 MB)

 Trainable params: 25,690,625 (98.00 MB)

 Non-trainable params: 42,658,176 (162.73 MB)

In [34]:
# Compile dan fit model ResNet101 (belum dirun karena lama)
model_res101.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

# Memulai waktu training
start_time = time.time()

# Training
history_res101 = model_res101.fit(
    train_generator_res,
    epochs=10,
    validation_data=validation_generator_res)

# Menghitung waktu training
end_time = time.time()
total_training_time = end_time - start_time
print(f"Total waktu training: {total_training_time:.2f} detik")

In [ ]:
# # Compile dan fit model ResNet101 (rmsprop)
# model_res101.compile(
#     loss='binary_crossentropy',
#     optimizer='rmsprop',
#     metrics=['accuracy']
#     )

# # Memulai waktu training
# start_time = time.time()

# # Training
# history_res101 = model_res101.fit(
#     train_generator_res,
#     epochs=10,
#     validation_data=validation_generator_res)

# # Menghitung waktu training
# end_time = time.time()
# total_training_time = end_time - start_time
# print(f"Total waktu training: {total_training_time:.2f} detik")

###### Inception V3

In [35]:
# Modelling menggunakan InceptionV3
incv3 = InceptionV3(input_shape=(299, 299, 3), weights='imagenet', include_top=False)
for layer in incv3.layers:
    layer.trainable = False

x = incv3.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

model_incv3 = Model(inputs=incv3.input, outputs=prediction)
model_incv3.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 55,357,729 (211.17 MB)

 Trainable params: 33,554,945 (128.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [36]:
# Compile dan fit model ResNet101
model_incv3.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

# Memulai waktu training
start_time = time.time()

# Training
history_incv3 = model_incv3.fit(
    train_generator_inc,
    epochs=10,
    validation_data=validation_generator_inc)

# Menghitung waktu training
end_time = time.time()
total_training_time = end_time - start_time
print(f"Total waktu training: {total_training_time:.2f} detik")

Epoch 1/10


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


103/103 ━━━━━━━━━━━━━━━━━━━━ 753s 7s/step - accuracy: 0.8323 - loss: 4.2234 - val_accuracy: 0.8854 - val_loss: 1.7821
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 772s 7s/step - accuracy: 0.9035 - loss: 1.8072 - val_accuracy: 0.9183 - val_loss: 1.0650
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 875s 8s/step - accuracy: 0.9298 - loss: 0.6253 - val_accuracy: 0.9305 - val_loss: 0.8046
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 680s 7s/step - accuracy: 0.9350 - loss: 0.4861 - val_accuracy: 0.9305 - val_loss: 0.3597
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 555s 5s/step - accuracy: 0.9426 - loss: 0.2818 - val_accuracy: 0.9341 - val_loss: 0.3015
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1122s 11s/step - accuracy: 0.9601 - loss: 0.1519 - val_accuracy: 0.9232 - val_loss: 0.3649
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 904s 9s/step - accuracy: 0.9521 - loss: 0.1673 - val_accuracy: 0.9268 - val_loss: 0.2903
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 907s 9s/step - accuracy: 0.9492 - loss: 0.1441 - val_accuracy: 0.9

In [1]:
# # Compile dan fit model ResNet101 (rmsprop)
# model_incv3.compile(
#     loss='binary_crossentropy',
#     optimizer='rmsprop',
#     metrics=['accuracy']
#     )

# # Memulai waktu training
# start_time = time.time()

# # Training
# history_incv3 = model_incv3.fit(
#     train_generator_inc,
#     epochs=10,
#     validation_data=validation_generator_inc)

# # Menghitung waktu training
# end_time = time.time()
# total_training_time = end_time - start_time
# print(f"Total waktu training: {total_training_time:.2f} detik")

In [37]:
model_incv3.save('incv3_model.h5')

### Test

In [3]:
import cv2

test_dir = os.path.join(base_dir, 'test')
test_images = [os.path.join(test_dir, img) for img in os.listdir(test_dir)]

test_data = []
for img_path in test_images:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (299, 299))
    img = img / 255.0
    test_data.append(img)

test_data = np.array(test_data)

# Prediksi
predictions = model_incv3.predict(test_data, verbose=1)

# Menampilkan gambar dan hasil prediksi
for img, pred in zip(test_data, predictions):
    plt.imshow(img)
    if pred > 0.5:
        plt.title("Pria")
    else:
        plt.title("Wanita")
    plt.axis('off')
    plt.show()